In [2]:
# required modules
import requests
import requests.exceptions
import re
from bs4 import BeautifulSoup
from collections import deque
import pandas as pd
import ethnicolr as ec
from time import sleep

# a queue of urls to be crawled
new_urls = deque(["https://uwaterloo.ca/school-of-accounting-and-finance/about/people",
                  "https://uwaterloo.ca/anthropology/about/people",
                  "https://uwaterloo.ca/renison/about/people",
                  "https://uwaterloo.ca/applied-mathematics/about/people",
                  "https://uwaterloo.ca/architecture/about/people",
                  "https://uwaterloo.ca/science/about/people",
                  "https://uwaterloo.ca/biology/about/people",
                  "https://uwaterloo.ca/biomedical-engineering/about/people",
                  "https://uwaterloo.ca/chemical-engineering/about/people",
                  "https://uwaterloo.ca/chemistry/about/people",
                  "https://uwaterloo.ca/civil-environmental-engineering/about/people",
                  "https://uwaterloo.ca/classical-studies/about/people",
                  "https://uwaterloo.ca/combinatorics-and-optimization/about/people/",
                  "https://uwaterloo.ca/communication-arts/about/people",
                  "https://cs.uwaterloo.ca/about/people",
                  "https://uwaterloo.ca/culture-and-language-studies/about/people",
                  "https://uwaterloo.ca/earth-environmental-sciences/about/people/",
                  "https://uwaterloo.ca/economics/about/people",
                  "https://uwaterloo.ca/electrical-computer-engineering/about/people",
                  "https://uwaterloo.ca/english/about/people",
                  "https://uwaterloo.ca/english-language-institute/about/people",
                  "https://uwaterloo.ca/school-environment-enterprise-development/people-profiles",
                  "https://uwaterloo.ca/environment-resources-and-sustainability/people-profiles",
                  "https://uwaterloo.ca/fine-arts/about/people",
                  "https://uwaterloo.ca/french-studies/about/people",
                  "https://uwaterloo.ca/geography-environmental-management/people-profiles",
                  "https://uwaterloo.ca/germanic-slavic-studies/about/people",
                  "https://uwaterloo.ca/stratford-school-of-interaction-design-and-business/about/people",
                  "https://uwaterloo.ca/history/about/people",
                  "https://uwaterloo.ca/jewish-studies/about/people",
                  "https://uwaterloo.ca/kinesiology/about/people/",
                  "https://uwaterloo.ca/knowledge-integration/about/people",
                  "https://uwaterloo.ca/management-sciences/about/people",
                  "https://uwaterloo.ca/mechanical-mechatronics-engineering/about/people/group",
                  "https://uwaterloo.ca/medieval-studies/about/people",
                  "https://uwaterloo.ca/grebel/people-profiles",
                  "https://uwaterloo.ca/nanotechnology/about/people",
                  "https://uwaterloo.ca/optometry-vision-science/about/people",
                  "https://uwaterloo.ca/peace-conflict-studies/about/people",
                  "https://uwaterloo.ca/pharmacy/about/people/group",
                  "https://uwaterloo.ca/philosophy/about/people",
                  "https://uwaterloo.ca/physics-astronomy/about/people",
                  "https://uwaterloo.ca/planning/about/people/group/54",
                  "https://uwaterloo.ca/planning/about/people/group/53",
                  "https://uwaterloo.ca/planning/about/people/group/56",
                  "https://uwaterloo.ca/political-science/about/people",
                  "https://uwaterloo.ca/psychology/about/people/",
                  "https://uwaterloo.ca/public-health-and-health-systems/about/people/",
                  "https://uwaterloo.ca/recreation-and-leisure-studies/about/people",
                  "https://uwaterloo.ca/religious-studies/about/people",
                  "https://uwaterloo.ca/aviation/about/people",
                  "https://uwaterloo.ca/science-and-business/about/people",
                  "https://uwaterloo.ca/social-development-studies/about/people",
                  "https://uwaterloo.ca/school-of-social-work/about/people",
                  "https://uwaterloo.ca/sociology-and-legal-studies/about/people",
                  "https://uwaterloo.ca/software-engineering/about/people",
                  "https://uwaterloo.ca/spanish/about/people",
                  "https://uwaterloo.ca/statistics-and-actuarial-science/about/people/",
                  "https://uwaterloo.ca/studies-in-islam/about/people",
                  "https://uwaterloo.ca/systems-design-engineering/about/people",
                  "https://uwaterloo.ca/theological-studies/about/people",
                  "https://uwaterloo.ca/womens-studies/about/people",
                  "https://uwaterloo.ca/centre-advanced-materials-joining/about/people/",
                  "https://uwaterloo.ca/bioengineering-biotechnology/about/people",
                  "https://uwaterloo.ca/groundwater-research/about/people",
                  "https://uwaterloo.ca/mental-health-research-treatment/about/people",
                  "https://uwaterloo.ca/centre-pavement-transportation-technology/about/people",
                  "https://uwaterloo.ca/centre-for-theoretical-neuroscience/about/people",
                  "https://uwaterloo.ca/conrad-school-entrepreneurship-business/about/people",
                  "https://uwaterloo.ca/games-institute/about/people",
                  "https://uwaterloo.ca/heritage-resources-centre/about/people",
                  "https://uwaterloo.ca/institute-computer-research/about/people",
                  "https://uwaterloo.ca/institute-polymer-research/about/people",
                  "https://uwaterloo.ca/climate-centre/about/people",
                  "https://uwaterloo.ca/southwestern-ontario-research-data-centre/about/people",
                  "https://uwaterloo.ca/survey-research-centre/about/people",
                  "https://uwaterloo.ca/water-institute/about/people",
                  "https://uwaterloo.ca/centre-automotive-research/about/people/",
                  "https://uwaterloo.ca/centre-for-german-studies/about/people",
                  "https://uwaterloo.ca/centre-advancement-co-operative-education/about/people",
                  "https://uwaterloo.ca/waterloo-institute-for-hellenistic-studies/about/people",
                  "https://uwaterloo.ca/institute-nanotechnology/about/people",
                  "https://uwaterloo.ca/waterloo-institute-for-social-innovation-and-resilience/about/people",
                  "https://uwaterloo.ca/waterloo-research-institute-in-insurance-securities-and-quantitative-finance/about/people"])
                  
# a set of urls that we have already crawled
processed_urls = set()

# dictionary to hold names and email addresses
data = {
    'First Name' : [],
    'Last Name' : [],
    'Email' : []
}

# process urls one by one until we exhaust the queue
while len(new_urls):
    # move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    processed_urls.add(url)
    
    # get url's content
    print("Processing %s" % url)
    try:
        response = requests.get(url)
        content = response.text
    except Exception as e:
        # ignore pages with errors
        print("\tERROR: " + e)
        continue

    # create a beutiful soup for the html document
    soup=BeautifulSoup(content,'lxml')
    
    # find div containers holding professor contact information
    profs = soup.find_all("div", id=re.compile("^node-"))
    for prof in profs:
        # obtain professor name
        name = prof.find("h2")
        if name == None or name == "TBD":
            continue
        else:
            name = name.get_text()
        # obtain professor email
        email = prof.find("a", href=re.compile(r"^mailto:"))
        if email == None:
            email = "No Email"
        else:
            email = email.get_text()
        # clean data and insert into dictionary
        if " (on leave)" in name.lower():
            name = name[:len(name)-11]
        if ", P.Eng." in name:
            name = name[:len(name)-8]
        if "Dr. " in name:
            name = name[4:]
        if "Dr " in name:
            name = name[3:]
        data['First Name'] += [name.split(" ")[0]]
        data['Last Name'] += [name.split(" ")[-1]]
        data['Email'] += [email]
    
    # pause scraping for 15 seconds (robots.txt asks for 12 second delay)
    sleep(15)
    
# create pandas dataframe and remove any duplicate entries
df = pd.DataFrame(data)
df = df.drop_duplicates()
    
# determine ethnicity
df = ec.pred_wiki_name(df,list(df)[1],list(df)[0])
    
# print dataframe and write data to csv file
print(df)
df.to_csv("waterloo parsed.csv")

Processing https://uwaterloo.ca/school-of-accounting-and-finance/about/people
Processing https://uwaterloo.ca/anthropology/about/people
Processing https://uwaterloo.ca/renison/about/people
Processing https://uwaterloo.ca/applied-mathematics/about/people
Processing https://uwaterloo.ca/architecture/about/people
Processing https://uwaterloo.ca/science/about/people
Processing https://uwaterloo.ca/biology/about/people
Processing https://uwaterloo.ca/biomedical-engineering/about/people
Processing https://uwaterloo.ca/chemical-engineering/about/people
Processing https://uwaterloo.ca/chemistry/about/people
Processing https://uwaterloo.ca/civil-environmental-engineering/about/people
Processing https://uwaterloo.ca/classical-studies/about/people
Processing https://uwaterloo.ca/combinatorics-and-optimization/about/people/
Processing https://uwaterloo.ca/communication-arts/about/people
Processing https://cs.uwaterloo.ca/about/people
Processing https://uwaterloo.ca/culture-and-language-studies/abo